In [1]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors import LengthBasedExampleSelector

# 1. 创建一组“反义词”任务的示例
examples = [
    {"input": "开心", "output": "伤心"},
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output": "没精打采"},
    {"input": "粗", "output": "细"},
]

example_prompt=PromptTemplate(
    input_variables=["input","output"],
    template="Input：{input}\nOutput：{output}"
)

example_selector=LengthBasedExampleSelector(
    examples=examples,              # 可供选择的示例。
    example_prompt=example_prompt,  # 用于格式化示例的提示词模板
    max_length=25,                  # 格式化后的示例最大字符长度
)

dynamic_prompt=FewShotPromptTemplate(
      example_selector=example_selector,  # 选择示例
      example_prompt=example_prompt,      # 示例提示词模板
      prefix="请将输入的词转换成反义词",    # 前缀
      suffix="Input：{adjective}\nOutput：",   # 后缀
      input_variables=["adjective"],          # 输入变量
)

In [2]:
print(dynamic_prompt.format(adjective="big"))

请将输入的词转换成反义词

Input：开心
Output：伤心

Input：高
Output：矮

Input：精力充沛
Output：没精打采

Input：粗
Output：细

Input：big
Output：


In [3]:
long_string="big and huge and massive and large and gigantic and tall and much much much much bigger and big "
print(dynamic_prompt.format(adjective=long_string))


请将输入的词转换成反义词

Input：开心
Output：伤心

Input：高
Output：矮

Input：big and huge and massive and large and gigantic and tall and much much much much bigger and big 
Output：


In [4]:
new_example={"input":"胖","output":"瘦"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="热情"))


请将输入的词转换成反义词

Input：开心
Output：伤心

Input：高
Output：矮

Input：精力充沛
Output：没精打采

Input：粗
Output：细

Input：胖
Output：瘦

Input：热情
Output：


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
openai_api_key="sk-Your OpenAI API Key"
openai_api_base="http://127.0.0.1:1234/v1"
chat=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key=openai_api_key,
  base_url=openai_api_base,
  temperature=0.7
  )

output_parser=StrOutputParser()

chain=dynamic_prompt | chat | output_parser

chain.invoke({"adjective":"热情"})

'当然，以下是将输入的词转换成反义词的结果：\n\nInput：开心 -> 不开心\nOutput：高 -> 矮\nInput：精力充沛 -> 没精打采\nOutput：粗 -> 细\nInput：胖 -> 肥瘦\nOutput：热情 -> 冷静'

最大余玄相似度的嵌入示例

In [6]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector,MaxMarginalRelevanceExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

# --- 第二部分：准备示例和选择器 ---
examples = [
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output": "没精打采"},
    {"input": "粗", "output": "细"},
    {"input": "快乐", "output": "悲伤"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

In [7]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,# 示例列表
    embeddings,# 用于计算相似度的嵌入模型
    FAISS, # 使用 FAISS 向量数据库在内存中存储
    k=2    # 每次只选出最相关的一个例子
)

# --- 第三部分：构建 FewShot 模板 ---
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请给出输入词的反义词",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"]
)

In [8]:
print(mmr_prompt.format(adjective="担心"))

请给出输入词的反义词

Input: 粗
Output: 细

Input: 快乐
Output: 悲伤

Input: 担心
Output:


通过n-gram重叠选择

In [9]:
from langchain_community.example_selectors import NGramOverlapExampleSelector

# 1. 定义示例：虚构的翻译任务
examples = [
    {"input": "See Spot run.", "output": "请参阅现场运行。"},
    {"input": "My dog barks.", "output": "我的狗吠叫。"},
    {"input": "Cat can run.", "output": "猫会跑。"},
]

In [10]:
# 3. 创建 NGramOverlap 选择器
# threshold: 阈值。如果设为 -1.0（默认），则按相似度排序选出最像的；
# 如果设为 0.0，则排除完全不重叠的。
example_selector = NGramOverlapExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    threshold=-1.0 
)

# 4. 构建 FewShot 模板
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="请将下列英文句子翻译成中文",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"]
)

In [11]:
print(dynamic_prompt.format(sentence="cat can run fast."))

请将下列英文句子翻译成中文

Input: Cat can run.
Output: 猫会跑。

Input: See Spot run.
Output: 请参阅现场运行。

Input: My dog barks.
Output: 我的狗吠叫。

Input: cat can run fast.
Output:


In [12]:
example_selector.threshold=0.0
print(dynamic_prompt.format(sentence="cat can run fast."))

请将下列英文句子翻译成中文

Input: Cat can run.
Output: 猫会跑。

Input: cat can run fast.
Output:


In [13]:
chain=dynamic_prompt | chat | output_parser
chain.invoke({"sentence":"cat can play fetch."})

'猫可以玩 Fetch。'